In [131]:
import struct
import pandas as pd
import yaml
from datetime import datetime 
pd.set_option('display.max_columns', None)
fn = '/Users/ruggles/repos/racing_data/data/limerock_mx5.ibt'

In [164]:
def read_sub_header(f):
    sub_header_dict = {}
    sub_header_dict['start_date'] =  datetime.fromtimestamp(struct.unpack('<q', f.read(8))[0]).isoformat()
    sub_header_dict['start_time'] =  struct.unpack('<d', f.read(8))[0]
    sub_header_dict['end_time'] =  struct.unpack('<d', f.read(8))[0]
    sub_header_dict['lap_count'] =  struct.unpack('<i', f.read(4))[0]
    sub_header_dict['record_count'] =  struct.unpack('<i', f.read(4))[0]
    #header length totals 32 bytes
    return sub_header_dict

def read_telemetry_header(f):
    telemetry_header_dict = {}
    telemetry_header_dict['version'] = struct.unpack('<i', f.read(4))[0]
    telemetry_header_dict['status'] = struct.unpack('<i', f.read(4))[0]
    telemetry_header_dict['tick_rate'] = struct.unpack('<i', f.read(4))[0]
    telemetry_header_dict['session_info_update'] = struct.unpack('<i', f.read(4))[0]
    telemetry_header_dict['session_info_length'] = struct.unpack('<i', f.read(4))[0]
    telemetry_header_dict['session_info_offset'] = struct.unpack('<i', f.read(4))[0]
    telemetry_header_dict['num_vars'] = struct.unpack('<i', f.read(4))[0]
    telemetry_header_dict['var_header_offset'] = struct.unpack('<i', f.read(4))[0]
    telemetry_header_dict['num_buf'] = struct.unpack('<i', f.read(4))[0]
    telemetry_header_dict['buf_len'] = struct.unpack('<i', f.read(4))[0]
    f.seek(16,1)
    telemetry_header_dict['buf_offset'] = struct.unpack('<i', f.read(4))[0]
    f.seek(52,1)
    #header length totals 112 bytes
    return telemetry_header_dict

def read_var_header(f):
    var_header_dict = {}
    var_header_dict['type'] = struct.unpack('<i', f.read(4))[0]
    var_header_dict['offset'] = struct.unpack('<i', f.read(4))[0]
    var_header_dict['count'] = struct.unpack('<i', f.read(4))[0]
    var_header_dict['count_as_time'] = struct.unpack('<b', f.read(1))[0]
    f.seek(3,1)
    var_header_dict['name'] = str(f.read(32)).replace('\\x00','').replace("b'","").replace("'","")
    var_header_dict['description'] = str(f.read(64)).replace('\\x00','').replace("b'","").replace("'","")
    var_header_dict['unit'] = str(f.read(32)).replace('\\x00','').replace("b'","").replace("'","")
    #header length totals 144 bytes
    return var_header_dict

def var_meta_data(variable_defs,parameter):
    var_data_list = []
    for var in variable_defs:
        var_data_list.append(var[parameter])
    return var_data_list

#old slow method
def variable_data(f,var_funcs):
    if var_type == 0:
        data = str(f.read(1)).replace('\\x00','').replace("b'","").replace("'","")
    elif var_type == 1:
        data = struct.unpack('<b', f.read(1))[0]
    elif var_type == 2:
        data = struct.unpack('<i', f.read(4))[0]
    elif var_type == 3:
        data = struct.unpack('<I', f.read(4))[0]
    elif var_type == 4:
        data = struct.unpack('<f', f.read(4))[0]
    elif var_type == 5:
        data = struct.unpack('<d', f.read(8))[0]
    return data

def variable_string_func(var_type):
    if var_type == 0:
        string_func = '''str(f.read(1)).replace('\\x00','').replace("b'","").replace("'","")'''
    elif var_type == 1:
        string_func = "struct.unpack('<b', f.read(1))[0]"
    elif var_type == 2:
        string_func = "struct.unpack('<i', f.read(4))[0]"
    elif var_type == 3:
        string_func = "struct.unpack('<I', f.read(4))[0]"
    elif var_type == 4:
        string_func = "struct.unpack('<f', f.read(4))[0]"
    elif var_type == 5:
        string_func = "struct.unpack('<d', f.read(8))[0]"
    return string_func

var_type_bytes = {0:1,1:1,2:4,3:4,4:4,5:8}
var_type_structs = {0:'c',1:'b',2:'i',3:'I',4:'f',5:'d'}

def create_var_funcs(variable_defs,white_list=None,black_list=[]):
    var_name_list = var_meta_data(variable_defs,'name')
    var_type_list = var_meta_data(variable_defs,'type')
    
    #get the desired column names
    if white_list == None:
        name_list = var_name_list
    else:
        name_list = white_list
    final_names = []
    for item in name_list:
        if item not in black_list:
            final_names.append(item)
    var_func_list = []
    var_func_skip = []
    skip_bytes = 0
    for x,var_name in enumerate(var_name_list):
        keep = True
        if var_name in final_names:
            if keep == False:
                var_func_list.append('f.seek({skip_bytes},1)'.format(skip_bytes=skip_bytes))
                var_func_skip.append(True)
                keep = True
            var_func_list.append(variable_string_func(var_type_list[x]))
            var_func_skip.append(False)
        else:
            skip_bytes+=var_type_bytes[var_type_list[x]]
                
    return {'column_names':final_names,'var_func_list':var_func_list,'var_func_skip':var_func_skip}

def create_struct_dict(variable_defs):
    struct_str = '<'
    struct_len = 0
    for item in variable_defs:
        struct_str+=var_type_structs[item['type']]
        struct_len+=var_type_bytes[item['type']]
    return {'struct_str':struct_str,'struct_len':struct_len}

def create_data_dict(fn):
    variable_defs = []
    full_data = []
    with open(fn, 'rb') as f:
        telemetry_header = read_telemetry_header(f)
        sub_header = read_sub_header(f)
        for i in range(telemetry_header['num_vars']):
            variable_defs.append(read_var_header(f))
        f.seek(telemetry_header['session_info_length'],1)
        struct_def = create_struct_dict(variable_defs)
        structobj = struct.Struct(struct_def['struct_str'])
        for r in range(sub_header['record_count']):
            
            
        ###slow text method
        #var_type_list = var_meta_data(variable_defs,'type')
        #var_name_list = var_meta_data(variable_defs,'name')
        #var_funcs = create_var_funcs(variable_defs)
        #var_func_list = var_funcs['var_func_list']
        #var_name_list = var_funcs['column_names']
        #var_func_skip = var_funcs['var_func_skip']
        #for r in range(sub_header['record_count']):
            #data_list = []
            #for x,var_func in enumerate(var_func_list):
            #    if var_func_skip[x]:
            #        eval(var_func)
            #    else:
            #        data_list.append(eval(var_func))
            ###faster method
            #data_list=[]
            #for var_type in var_type_list:
            #    data_list.append(variable_data(f,var_type))
            full_data.append(structobj.unpack(f.read(struct_def['struct_len'])))
    return {'data':full_data, 'headers':var_name_list,'telemetry_header':telemetry_header,'sub_header':sub_header,'variable_defs':variable_defs}

def get_session_data(fn,data_dict):
    string_yaml = ""
    with open(fn,'rb') as f:
        #f.seek(data_dict['telemetry_header']['session_info_offset'])
        #file_segment = f.read(data_dict['telemetry_header']['session_info_length'])
        #for x,line in enumerate(file_segment):
        #    string_yaml = string_yaml + str(line).replace("b'","").replace("'","").replace('\\n','').replace('\\','') + '\n'
        for x,line in enumerate(f):
            if line == b'...\n':
                break
            if x >=1:
                string_yaml = string_yaml + str(line).replace("b'","").replace("'","").replace('\\n','').replace('\\','') + '\n'
        session_dict = yaml.safe_load(string_yaml)
    return session_dict

In [165]:
data_dict = create_data_dict(fn)
session_dict = get_session_data(fn,data_dict)
df = pd.DataFrame(data_dict['data'])
df.columns = data_dict['headers']

In [166]:
df

,SessionTime,SessionTick,SessionNum,SessionState,SessionUniqueID,SessionFlags,SessionTimeRemain,SessionLapsRemain,SessionLapsRemainEx,SessionTimeTotal,SessionLapsTotal,SessionJokerLapsRemain,SessionOnJokerLap,SessionTimeOfDay,DriverMarker,PushToPass,ManualBoost,ManualNoBoost,IsOnTrack,FrameRate,CpuUsageFG,GpuUsage,ChanAvgLatency,ChanLatency,ChanQuality,ChanPartnerQuality,CpuUsageBG,ChanClockSkew,MemPageFaultSec,PlayerCarPosition,PlayerCarClassPosition,PlayerCarClass,PlayerTrackSurface,PlayerTrackSurfaceMaterial,PlayerCarIdx,PlayerCarTeamIncidentCount,PlayerCarMyIncidentCount,PlayerCarDriverIncidentCount,PlayerCarWeightPenalty,PlayerCarPowerAdjust,PlayerCarDryTireSetLimit,PlayerCarTowTime,PlayerCarInPitStall,PlayerCarPitSvStatus,PlayerTireCompound,PlayerFastRepairsUsed,PaceMode,OnPitRoad,SteeringWheelAngle,Throttle,Brake,Clutch,Gear,RPM,Lap,LapCompleted,LapDist,LapDistPct,LapBestLap,LapBestLapTime,LapLastLapTime,LapCurrentLapTime,LapLasNLapSeq,LapLastNLapTime,LapBestNLapLap,LapBestNLapTime,LapDeltaToBestLap,LapDeltaToBestLap_DD,LapDeltaToBestLap_OK,LapDeltaToOptimalLap,LapDeltaToOptimalLap_DD,LapDeltaToOptimalLap_OK,LapDeltaToSessionBestLap,LapDeltaToSessionBestLap_DD,LapDeltaToSessionBestLap_OK,LapDeltaToSessionOptimalLap,LapDeltaToSessionOptimalLap_DD,LapDeltaToSessionOptimalLap_OK,LapDeltaToSessionLastlLap,LapDeltaToSessionLastlLap_DD,LapDeltaToSessionLastlLap_OK,Speed,Yaw,YawNorth,Pitch,Roll,EnterExitReset,Lat,Lon,Alt,TrackTemp,TrackTempCrew,AirTemp,WeatherType,Skies,AirDensity,AirPressure,WindVel,WindDir,RelativeHumidity,FogLevel,PitsOpen,PitRepairLeft,PitOptRepairLeft,PitstopActive,FastRepairUsed,FastRepairAvailable,LFTiresUsed,RFTiresUsed,LRTiresUsed,RRTiresUsed,LeftTireSetsUsed,RightTireSetsUsed,FrontTireSetsUsed,RearTireSetsUsed,TireSetsUsed,LFTiresAvailable,RFTiresAvailable,LRTiresAvailable,RRTiresAvailable,LeftTireSetsAvailable,RightTireSetsAvailable,FrontTireSetsAvailable,RearTireSetsAvailable,TireSetsAvailable,IsOnTrackCar,SteeringWheelPctTorque,SteeringWheelPctTorqueSign,SteeringWheelPctTorqueSignStops,SteeringWheelPctDamper,SteeringWheelAngleMax,SteeringWheelLimiter,ShiftIndicatorPct,ShiftPowerPct,ShiftGrindRPM,ThrottleRaw,BrakeRaw,HandbrakeRaw,SteeringWheelMaxForceNm,SteeringWheelUseLinear,BrakeABSactive,EngineWarnings,FuelLevel,FuelLevelPct,PitSvFlags,PitSvLFP,PitSvRFP,PitSvLRP,PitSvRRP,PitSvFuel,PitSvTireCompound,TireLF_RumblePitch,TireRF_RumblePitch,TireLR_RumblePitch,TireRR_RumblePitch,SteeringWheelTorque,VelocityZ,VelocityY,VelocityX,YawRate,PitchRate,RollRate,VertAccel,LatAccel,LongAccel,dcStarter,dpRFTireChange,dpLFTireChange,dpRRTireChange,dpLRTireChange,dpFuelFill,dpWindshieldTearoff,dpFuelAddKg,dpFastRepair,dpLFTireColdPress,dpRFTireColdPress,dpLRTireColdPress,dpRRTireColdPress,WaterTemp,WaterLevel,FuelPress,FuelUsePerHour,OilTemp,OilPress,OilLevel,Voltage,ManifoldPress,RRbrakeLinePress,RRspeed,RRpressure,RRcoldPressure,RRtempL,RRtempM,RRtempR,RRtempCL,RRtempCM,RRtempCR,RRwearL,RRwearM,RRwearR,LRbrakeLinePress,LRspeed,LRpressure,LRcoldPressure,LRtempL,LRtempM,LRtempR,LRtempCL,LRtempCM,LRtempCR,LRwearL,LRwearM,LRwearR,RFbrakeLinePress,RFspeed,RFpressure,RFcoldPressure,RFtempL,RFtempM,RFtempR,RFtempCL,RFtempCM,RFtempCR,RFwearL,RFwearM,RFwearR,LFbrakeLinePress,LFspeed,LFpressure,LFcoldPressure,LFtempL,LFtempM,LFtempR,LFtempCL,LFtempCM,LFtempCR,LFwearL,LFwearM,LFwearR,RRshockDefl,RRshockVel,LRshockDefl,LRshockVel,RFshockDefl,RFshockVel,LFshockDefl,LFshockVel,LFrideHeight,RFrideHeight,LRrideHeight,RRrideHeight
0,37.749999,2326,0,4,1,268435968,604800.0,32767,32767,604800.0,32767,0,0,54397.0,0,0,0,0,1,77.860420,0.267013,0.294937,0.0,0.0,0.0,0.0,0.095,0.0,0.0,0,0,0,-1,-1,0,0,0,0,0.0,0.0,0,0.0,0,0,0,0,4,0,-0.025132,0.000000e+00,1.0,0.0,0,300.000000,0,0,0.000000,0.000000,0,0.0,0.0,0.000000,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,45.0,45.0,24.442816,3,0,1.157097,29.376572,0.89408,2.980232e-08,0.550057,0.0,1,0.0,0.0,0,0

In [156]:
data = b'/x01a/x02/x02/x01'

In [159]:
structobj = struct.Struct("<bbbh")
with open(fn,'rb') as f:
    fred = structobj.unpack(f.read(5))

In [160]:
fred

(2, 0, 0, 256)

In [153]:
data

b'/x00'

In [148]:
data_dict['sub_header']

{'start_date': '2022-04-28T20:28:29',
 'start_time': 37.7499994913734,
 'end_time': 156.16666615805735,
 'lap_count': 2,
 'record_count': 7098}

In [150]:
data_dict['variable_defs']

[{'type': 5,
  'offset': 0,
  'count': 1,
  'count_as_time': 0,
  'name': 'SessionTime',
  'description': 'Seconds since session start',
  'unit': 's'},
 {'type': 2,
  'offset': 8,
  'count': 1,
  'count_as_time': 0,
  'name': 'SessionTick',
  'description': 'Current update number',
  'unit': ''},
 {'type': 2,
  'offset': 12,
  'count': 1,
  'count_as_time': 0,
  'name': 'SessionNum',
  'description': 'Session number',
  'unit': ''},
 {'type': 2,
  'offset': 16,
  'count': 1,
  'count_as_time': 0,
  'name': 'SessionState',
  'description': 'Session state',
  'unit': 'irsdk_SessionState'},
 {'type': 2,
  'offset': 20,
  'count': 1,
  'count_as_time': 0,
  'name': 'SessionUniqueID',
  'description': 'Session ID',
  'unit': ''},
 {'type': 3,
  'offset': 24,
  'count': 1,
  'count_as_time': 0,
  'name': 'SessionFlags',
  'description': 'Session flags',
  'unit': 'irsdk_Flags'},
 {'type': 5,
  'offset': 28,
  'count': 1,
  'count_as_time': 0,
  'name': 'SessionTimeRemain',
  'description': 

In [151]:
data_dict['telemetry_header']

{'version': 2,
 'status': 1,
 'tick_rate': 60,
 'session_info_update': 0,
 'session_info_length': 12181,
 'session_info_offset': 36288,
 'num_vars': 251,
 'var_header_offset': 144,
 'num_buf': 1,
 'buf_len': 967,
 'buf_offset': 0}

# Notes
Useful Links
- https://python.readthedocs.io/en/stable/library/struct.html
- https://github.com/SkippyZA/ibt-telemetry
- https://forums.iracing.com/discussion/62/iracing-sdk/p1